In [21]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

def load_documents():
    document_loader = PyPDFDirectoryLoader("data/")
    return document_loader.load()

In [22]:
documents = load_documents()

In [23]:
print(documents[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}


In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents : list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100,
        length_function = len,
        is_separator_regex = False,
    )
    
    return text_splitter.split_documents(documents)


In [25]:
chunks = split_documents(documents)
print(chunks[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}


In [12]:

from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [17]:
#run this if you want to delte the already created chroma database
'''
import os
import shutil
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)
'''


In [5]:
from langchain.vectorstores.chroma import Chroma
CHROMA_PATH = "chroma"

In [27]:
#run this if chroma database is not already created
'''
db = Chroma.from_documents(
    chunks,embeddings_model,persist_directory=CHROMA_PATH
)
'''

In [13]:
#connecting to existing database
db = Chroma(persist_directory=CHROMA_PATH,embedding_function=embeddings_model)

In [63]:
from langchain.prompts import ChatPromptTemplate


In [71]:
query = "i have a runny nose and a sore throat."

In [72]:
results  = db.similarity_search_with_relevance_scores(query,k=10)
context_text = '\n\n---\n\n'.join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context = context_text,question=query)
#print(context_text)

In [69]:
PROMPT_TEMPLATE ="""
Answer the questio based only on the following context:
{context}

---
Answer the question based on the above context:{question}
"""

In [73]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(model="llama2")
response_text = llm.invoke(prompt)
print(response_text)

Based on the information provided in the passage, it is possible that you have a viral infection such as the common cold or a virus like adenovirus or coxsackie virus, which can cause both a runny nose and a sore throat. However, without further testing or evaluation by a medical professional, it is difficult to determine the exact cause of your symptoms. It is important to consult with a doctor to rule out any serious underlying conditions and to receive proper treatment for your symptoms.
